In [10]:
# Useful starting lines
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
%load_ext autoreload
%autoreload 2

In [ ]:
end_data = 5
is_10K = False
fname = '1_metadata_stat_extract_3'
smooth_nbr = 1
figure_name=('basic_mode_new_%s' %smooth_nbr)
train_epoch = []
train_cu_loss = []
train_avg_loss = []
train_cu_acc = []
train_avg_acc = []
train_avg_hard_acc = []
train_avg_indi_acc = []

test_epoch = []
test_cu_loss = []
test_avg_loss = []
test_cu_acc = []
test_avg_acc = []
test_avg_hard_acc = []
test_avg_indi_acc = []

train_epoch, train_avg_acc, train_avg_loss, train_avg_indi_acc, test_epoch, test_avg_acc, test_avg_loss, test_avg_indi_acc = create_data(fname, end_data,is_10K, smooth_nbr=smooth_nbr)

create_figure(train_epoch, train_avg_acc, train_avg_loss, train_avg_indi_acc, test_epoch, test_avg_acc, test_avg_loss, test_avg_indi_acc,name=figure_name, is_10K=is_10K)

In [27]:
def extract_results(fname, end_data_data, is_10k):
    
    
    if(is_10K):
        duplicate = 3
    else:
        duplicate = 1
    train_name = (('EDMN-theano/output_data/train_babi%s.csv' %fname))
    test_name = (('EDMN-theano/output_data/test_babi%s.csv' %fname))
    with open(train_name) as f:
        reader = csv.reader(f)
        i=0
        for l in reader:
            i = i + 1
            if(len(l)>=6):
                e = float(l.pop(0))
                if(e<=end_data):# and i%10==0):
                    train_epoch.append(e*10000 + float(l.pop(0)))
                    train_cu_loss.append(l.pop(0))
                    avg_l = float(l.pop(0))
                    train_avg_loss.append(avg_l)
                    train_cu_acc.append(l.pop(0)) #softacc_cu
                    train_avg_acc.append(l.pop(0)) #avg_soft_acc
                    temp = l.pop(0)
                    train_avg_hard_acc.append(train_cu_acc[-1])
                    train_avg_indi_acc.append(l.pop(0))
    with open(test_name) as f:
        reader = csv.reader(f)
        for l in reader:
            if(len(l)>=6):
                t = float(l.pop(0))
                if(t<=end_data):
                    temp = []
                    temp.append(l.pop(0))
                    temp.append(l.pop(0))
                    temp.append(l.pop(0))
                    temp.append(l.pop(0))
                    temp.append(l.pop(0))
                    temp.append(l.pop(0))
                    temp.append(l.pop(0))
                    for i in range(0,duplicate):
                        test_epoch.append(t*1000 + float(temp[0]))
                        test_cu_loss.append(temp[1])
                        test_avg_loss.append(temp[2])
                        test_cu_acc.append(temp[3])
                        test_avg_acc.append(temp[4])
                        test_avg_hard_acc.append(temp[3])
                        test_avg_indi_acc.append(temp[6])
    return train_epoch, train_avg_acc, train_avg_loss, train_avg_indi_acc, test_epoch, test_avg_acc, test_avg_loss, test_avg_indi_acc

In [12]:
import csv
def create_data(fname, end_data,is_10k, smooth_nbr=1):

    train_epoch, train_avg_acc, train_avg_loss, train_avg_indi_acc, test_epoch, test_avg_acc, test_avg_loss, test_avg_indi_acc = extract_results(fname, end_data, is_10K)
    
    test_avg_acc = test_avg_acc[0:len(train_avg_acc)]
    test_avg_loss = test_avg_loss[0:len(train_avg_loss)]

    train_avg_acc = smoother(train_avg_acc, smooth_nbr)[:len(train_avg_acc) - smooth_nbr]
    test_avg_acc = smoother(test_avg_acc, smooth_nbr)[:len(test_avg_acc) - smooth_nbr]
    train_avg_loss = smoother(train_avg_loss, smooth_nbr)[:len(train_avg_loss) - smooth_nbr]
    test_avg_loss = smoother(test_avg_loss, smooth_nbr)[:len(test_avg_loss) - smooth_nbr]
    train_epoch = train_epoch[:len(train_epoch) - smooth_nbr]
    
    #train_avg_indi_acc
    

    return train_epoch, train_avg_acc, train_avg_loss, train_avg_indi_acc, test_epoch, test_avg_acc, test_avg_loss, test_avg_indi_acc

In [13]:
def create_figure(train_epoch, train_avg_acc, train_avg_loss, train_avg_indi_acc, test_epoch, test_avg_acc, test_avg_loss, test_avg_indi_acc, name='', is_10K=False):
    plt.figure()
    
    

    plt.plot(train_epoch, train_avg_acc, label="training accuracy")
    plt.plot(train_epoch, test_avg_acc, label="testing accuracy")
    plt.legend()
    plt.xlabel("step number")
    plt.ylabel("average accuracy")
    if(is_10K):
        plt.title("Evolution of training and testing accuracy over time (10k)")
    else:
        plt.title("Evolution of training and testing accuracy over time")
    temp = ('results/avg_acc_%s.jpg' %name)
    plt.savefig(temp)


    plt.figure()

    plt.plot(train_epoch, train_avg_loss, label="training loss")
    plt.plot(train_epoch, test_avg_loss, label="testing loss")
    plt.legend()
    plt.xlabel("step number")
    plt.ylabel("average loss")
    if(is_10K):
        plt.title("Evolution of training and testing loss over time (10k)")
    else:
        plt.title("Evolution of training and testing loss over time")
    temp = ('results/avg_loss_%s.jpg' %name)
    plt.savefig(temp)

In [8]:
def smoother(l, nbr=2):
    new_l = l
    for i in range(0,np.shape(l)[0]-nbr+1):
        new_l[i] = float(l[i])
        for j in range(1, nbr):
            new_l[i] = new_l[i] + float(l[i+j])
        new_l[i] = new_l[i]/nbr
    return new_l